In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.6 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00


In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.5 MB/s eta 0:00:00


In [ ]:
!pip install flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from evaluate import load
from flask import request, jsonify
import jsonlines
import torch

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("laschulz/t5-large")
model = T5ForConditionalGeneration.from_pretrained("laschulz/t5-large")

In [ ]:
def extract_jsonl(file_path):
    data = []
    with jsonlines.open(file_path) as reader:
        for item in reader:
            data.append(item)
    return data

In [ ]:
database = extract_jsonl("./test_socratic.jsonl")

In [ ]:
def parser_expectedAnswer(expected_answer):
    temp = str(expected_answer).split('\n') #str might be redundant
    temp = [line.split('**')[0] for line in temp]
    if '###' in temp[-1]:
        temp.pop()
    return temp

In [ ]:
def parser_output(model_output):
    parsed_output = model_output.split('?')
    #parsed_output = [entry.strip() + '?' if entry.strip() != "" else "" for entry in model_output.split('?')]
    for i in range(len(parsed_output)):
        entry = parsed_output[i]
        stripped = entry.strip()
        if (stripped != ""):
            parsed_output[i] = stripped + '?'
        else:
            parsed_output.pop(i)
    return parsed_output

In [ ]:
bertscore = load("bertscore")
def bert_score(references, predictions): 
    results = bertscore.compute(predictions=predictions, references=references, lang="en")
    return results['f1']

In [ ]:
def process_database(database):
    all_middle = 0
    all_lower = 0
    some_lower = 0
    last_lower = 0
    all_upper = 0
    some_upper = 0
    first_upper = 0
    both = 0

    for item in database:
        input_text = item["question"]
        expected_answer = item["answer"]
        expected_answer = parser_expectedAnswer(expected_answer)
        inputs = tokenizer([input_text], return_tensors="pt", padding=True)

        # Check if a GPU is available
        if torch.cuda.is_available():
            # Set the device to GPU
            device = torch.device("cuda")
        else:
            # If GPU is not available, fall back to CPU
            device = torch.device("cpu")

        # Move the model and inputs to the GPU
        model.to(device)
        inputs = inputs.to(device)

        output_sequence = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            do_sample=False,
        )
        decoded_output = tokenizer.batch_decode(output_sequence, skip_special_tokens=True)

        parsed_output = parser_output(decoded_output[0])

        if (len(parsed_output) == len(expected_answer)):
            bertScore = bert_score(expected_answer,parsed_output)

            if all(0.8925 < score < 0.9925 for score in bertScore):
                all_middle += 1
            if all(score < 0.8925 for score in bertScore):
                all_lower += 1
            elif any(score < 0.8925 for score in bertScore):
                some_lower += 1
            if all(score > 0.9925 for score in bertScore):
                all_upper += 1
            elif any(score > 0.9925 for score in bertScore):
                some_upper += 1
            if bertScore[(len(bertScore)-1)] < 0.8925:
                last_lower += 1
            if bertScore[(len(bertScore)-1)] > 0.9925:
                first_upper += 1

            if any(score > 0.9925 for score in bertScore) and any(score < 0.8925 for score in bertScore):
                both += 1

            
    return (all_middle, all_lower, some_lower, last_lower, all_upper, some_upper, first_upper, both)
  

all_middle, all_lower, some_lower, last_lower, all_upper, some_upper, first_upper, both = process_database(database)

In [ ]:
print(all_middle, all_lower, some_lower, last_lower, all_upper, some_upper, first_upper, both)

191 2 69 66 33 119 81 19


In [ ]:
def process_database(database):
    results = []
    raw_results = []
    length_is_not_the_same = 0
    length_is_same = 0
    over_generation = 0
    question_missing = 0
    for item in database:
        input_text = item["question"]
        expected_answer = item["answer"]
        expected_answer = parser_expectedAnswer(expected_answer)
        inputs = tokenizer([input_text], return_tensors="pt", padding=True)

        # Check if a GPU is available
        if torch.cuda.is_available():
            # Set the device to GPU
            device = torch.device("cuda")
        else:
            # If GPU is not available, fall back to CPU
            device = torch.device("cpu")

        # Move the model and inputs to the GPU
        model.to(device)
        inputs = inputs.to(device)

        output_sequence = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            do_sample=False,
        )
        decoded_output = tokenizer.batch_decode(output_sequence, skip_special_tokens=True)

        parsed_output = parser_output(decoded_output[0])

        if (len(parsed_output) != len(expected_answer)):
            length_is_not_the_same += 1
            if (len(parsed_output) > len(expected_answer)):
                over_generation += 1
            else:
                question_missing += 1
        else:
            length_is_same += 1
            print("same: ", length_is_same)
            bertScore = bert_score(expected_answer,parsed_output)
            result_dict = {
                "bert_score": bertScore,
                "model_output": parsed_output,
                "expected_answer": expected_answer
            }
            for i in range(len(bertScore)):
                score = bertScore[i]
                if (bertScore[i] > 0.995):
                  break;
                if (i == len(bertScore)-1):
                    raw_results_dict = { 
                        "question": item["question"],
                        "answer": item["answer"]
                    }
                    raw_results.append(raw_results_dict)
                    print("parsed_output: ", parsed_output)
                    print("expected_outut: ", expected_answer)

            results.append(result_dict)
            
    return (results, raw_results, length_is_not_the_same, over_generation, question_missing)

In [ ]:
def process_database(database):
    raw_results_good = []
    raw_results_bad = []
    length_is_not_the_same = 0
    length_is_same = 0
    for item in database:
        input_text = item["question"]
        expected_answer = item["answer"]
        expected_answer = parser_expectedAnswer(expected_answer)
        inputs = tokenizer([input_text], return_tensors="pt", padding=True)

        # Check if a GPU is available
        if torch.cuda.is_available():
            # Set the device to GPU
            device = torch.device("cuda")
        else:
            # If GPU is not available, fall back to CPU
            device = torch.device("cpu")

        # Move the model and inputs to the GPU
        model.to(device)
        inputs = inputs.to(device)

        output_sequence = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            do_sample=False,
        )
        decoded_output = tokenizer.batch_decode(output_sequence, skip_special_tokens=True)

        parsed_output = parser_output(decoded_output[0])

        if (len(parsed_output) != len(expected_answer)):
            length_is_not_the_same += 1
 
        else:
            length_is_same += 1
            print("same: ", length_is_same)
            bertScore = bert_score(expected_answer,parsed_output)
            result_dict = {
                "bert_score": bertScore,
                "model_output": parsed_output,
                "expected_answer": expected_answer
            }
            for i in range(len(bertScore)):
                score = bertScore[i]
                #looking at cases where there is a correct answer
                if (bertScore[i] > 0.995):
                  raw_results_dict = { 
                        "question": item["question"],
                        "answer": item["answer"]
                  }
                  raw_results_good.append(raw_results_dict)
                  break;
            
            for i in range(len(bertScore)):
                score = bertScore[i]
                #looking at cases where there is a wrong answer
                if (bertScore[i] < 0.90):
                  raw_results_dict = { 
                        "question": item["question"],
                        "answer": item["answer"]
                  }
                  raw_results_bad.append(raw_results_dict)
                  break;
            
    return (raw_results_good, raw_results_bad, length_is_not_the_same)

In [ ]:
raw_results_good, raw_results_bad, length_is_not_the_same = process_database(database)

In [ ]:
print(length_is_not_the_same)

In [ ]:
import jsonlines

def save_results_to_jsonl(results, filename):
    with jsonlines.open(filename, 'w') as writer:
        for result in results:
            writer.write(result)

In [ ]:
save_results_to_jsonl(results_analysis, 'results.jsonl')

In [ ]:
save_results_to_jsonl(raw_results_good, 'filtered_test_good.jsonl')

In [ ]:
save_results_to_jsonl(raw_results_bad, 'filtered_test_bad.jsonl')

In [ ]:
save_results_to_jsonl(raw_results[1:50], 'filtered_test_1-50.jsonl')

In [ ]:
print(len(raw_results_good))
print(len(raw_results_bad))